# CONSTRAINT SATISFACTION PROBLEMS (CSP)

This IPy notebook uses of the implementations in **csp.py** module provided in the supporting materials of the book* Artificial Intelligence: A Modern Approach*.

In [ ]:
from csp import *
# from notebook import psource, plot_NQueens

# %matplotlib inline
# Hide warnings in the matplotlib sections

import math
import warnings
warnings.filterwarnings("ignore")

## Zebra puzzle
Solves an instance of the "zebra puzzle".<br>
You have to try several times until you get a solution.

`Exercise:` Change the **min_conflicts** solve_zebra to get allways a solution. 

In [ ]:
# Solve Zebra using min_conflicts
# Please, run several trials until you do not get an error

solve_zebra(algorithm=min_conflicts, max_steps=10000)

In [ ]:
# Solve Zebra using backtracking search

solve_zebra(algorithm=backtracking_search, 
            select_unassigned_variable=mrv, order_domain_values=lcv, inference=forward_checking)

In [ ]:
# Solve Zebra using ac_sover

# variables
Colors = 'Red Yellow Blue Green Ivory'.split()
Pets = 'Dog Fox Snails Horse Zebra'.split()
Drinks = 'OJ Tea Coffee Milk Water'.split()
Countries = 'Englishman Spaniard Norwegian Ukranian Japanese'.split()
Smokes = 'Kools Chesterfields Winston LuckyStrike Parliaments'.split()
variables = set (Colors + Pets + Drinks + Countries + Smokes)

# Domains
dominio = {}
for var in variables:
    dominio[var] = set(range(1, 6))     # list(range(1, 6))
dominio['Norwegian'] = {1}
dominio['Milk'] = {3}

# Constraints
restricoes = [
              Constraint(Colors, all_diff_constraint),
              Constraint(Pets, all_diff_constraint),
              Constraint(Drinks, all_diff_constraint),
              Constraint(Countries, all_diff_constraint),
              Constraint(Smokes, all_diff_constraint),
              Constraint(('Englishman', 'Red'), eq),
              Constraint(('Spaniard', 'Dog'), eq),
              Constraint(('Kools', 'Yellow'), eq),
              Constraint(('Winston', 'Snails'), eq),
              Constraint(('LuckyStrike', 'OJ'), eq),
              Constraint(('Ukranian', 'Tea'), eq),
              Constraint(('Japanese', 'Parliaments'), eq),
              Constraint(('Coffee', 'Green'), eq),
              Constraint(('Chesterfields', 'Fox'), lambda a, b: abs(a - b) == 1),
              Constraint(('Norwegian', 'Blue'), lambda a, b: abs(a - b) == 1),
              Constraint(('Kools', 'Horse'), lambda a, b: abs(a - b) == 1),
              Constraint(('Green', 'Ivory'), lambda a, b: b == a - 1),
              ]

# days_too_short
zebra_csp = NaryCSP(dominio, restricoes)
print(zebra_csp.variables)

# Apply solver
ans = ac_solver(zebra_csp, arc_heuristic=sat_up)

# Print result
for h in range(1, 6):
        print('House', h, end=' ')
        for (var, val) in ans.items():
            if val == h:
                print(var, end=' ')
        print()

## cryptarithmetic problem

Each letter stands for a distinct digit; the aim is to find a substitution of digits for letters such that the resulting sum is arithmetically correct, with the added restriction that no leading zeroes are allowed.

**Puzzle D A Y S + T O O = S H O R T**

In [ ]:
# DAYS + TOO = SHORT 
# Solve the CSP using NARY_CSP + AC_Sover

# domain
dominio = {
          'D': set(range(1, 10)), 'A': set(range(0, 10)), 'Y': set(range(0, 10)), 'S': set(range(1, 10)),
          'T': set(range(1, 10)), 'O': set(range(0, 10)), 'H': set(range(0, 10)), 'R': set(range(0, 10)),
          'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2)), 'C4': set(range(0, 2))
          }

# constraints
restricoes = [
              Constraint(('D', 'A', 'Y', 'S', 'T', 'O', 'H', 'R'), all_diff_constraint),
              Constraint(('S', 'O', 'T', 'C1'), lambda s, o, t, c1: s + o == t + 10 * c1),
              Constraint(('Y', 'O', 'R', 'C1', 'C2'), lambda y, o, r, c1, c2: c1 + y + o == r + 10 * c2),
              Constraint(('A', 'T', 'O', 'C2', 'C3'), lambda a, t, o, c2, c3: c2 + a + t == o + 10 * c3),
              Constraint(('D', 'H', 'C3', 'C4'), lambda d, h, c3, c4: c3 + d == h + 10 * c4),
              Constraint(('S', 'C4'), eq)
              ]

# days_too_short
days_too_short = NaryCSP(dominio, restricoes)

# print(days_too_short.variables)
# print(days_too_short.domains)

# Result
ac_solver(days_too_short, arc_heuristic=sat_up)

In [ ]:
# D A Y S + TOO = SHORT 
# Solve the CSP using backtracking (depth-first search)

# CSP Definition
def days_too_short():
    Vars = 'A D H O R S T Y'.split()
    Conds = 'C1 C2 C3 C4'.split()
    variables = Vars + Conds
    #
    domains = {}
    for var in Vars:
        domains[var] = list(range(0, 10))
    for var in Conds:
        domains[var] = list(range(0, 2))  
    domains['D'] = domains['T'] = list(range(1, 10))
    domains['S'] = [1]
    #
    neighbors = parse_neighbors("""A: D Y S T O H R; D: Y S T O H R; Y: S T O H R;
                                S: T O H R; T: O H R; O: H R; H: R; S: C4""")

    # Not complete: Need to add the neighbors corresponding to the n_ary constraints
    # """C1: S O T Y R; C2: C1 O R Y; C3: C2 A O T; C4: C3 D H""")

    print(domains)
    print(neighbors)

    def dts_constraint(A, a, B, b, recurse=0):
        same = (a == b)
        if A == 'S' and B == 'C4':
            return same
        if recurse == 0:
            return dts_constraint(B, b, A, a, 1)
        if A in Vars and B in Vars:
            return not same

        raise Exception('error')

    return CSP(variables, domains, neighbors, dts_constraint)

# Call solver
dts = days_too_short()
backtracking_search(dts, select_unassigned_variable=mrv, order_domain_values=lcv, inference=forward_checking)